In [14]:
import requests
# from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree, html
import numpy as np
import var_path
from selenium import webdriver
from itertools import islice
import string
import re

##### general functions

# 1. get var from list of xpath
def get_var_xpath_from_list(url, item_list):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    # var = tree.xpath(var_xpath)
    variables = []
    for item in item_list:
        var = tree.xpath(item)
        variables.append(var)

    return variables

def get_var_xpath(url, xpath):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    var = tree.xpath(xpath)

    return var

def take(n, iterable):
    """ return n index of dictionary"""
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))

def merge(list1, list2):
 
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
     
    return merged_list

In [2]:
def get_url_series_year():
    """
    year_link = link of each series(year) 
    i.e. series 1877-2023

    select period of year in this def

    want to have 100 year 

    choose >> year_link[0:100] 

    all year = 133 years
    """

    url_list_year = 'https://www.espncricinfo.com/records/list-of-match-results-by-year-307847'
    url_espn = 'https://www.espncricinfo.com/'
    x_path_list_year = '//a[contains(@href,"/records/year/team-match-results")]/span/text()'
    xpath_get_attr = '//div[@class="ds-mb-6"]/a/@href'

    page = requests.get(url_list_year)
    tree = html.fromstring(page.content)
    var = tree.xpath(x_path_list_year)
    # print(var)

    years = get_var_xpath(url_list_year, x_path_list_year)
    # print(len(years))
    link = get_var_xpath(url_list_year, xpath_get_attr)
    # print(year_link)
    year_link = []
    for item in link:
        url = url_espn + item
        year_link.append(url)

    return year_link

year_link = get_url_series_year()

In [18]:
# //span/a[contains(@href, "/series/")]

url_year = ['https://www.espncricinfo.com/records/year/team-match-results/1877-1877/test-matches-1']

for url in url_year:
    var = get_var_xpath_from_list(url, var_path.team1_2_xp)
    # print(var)

    ## 1. get info from each year
    info_list_ea_year = []
    for i, item in enumerate(var[1]):
        if item.startswith('Test #'):
            sublist = var[0][i * 5:(i + 1) * 5]
            sublist.append(item)
            info_list_ea_year.append(sublist)
    # print(info_list_ea_year)

    ## 2. get url scorecard/summary and info from it
    url_espn = 'https://www.espncricinfo.com/'
    scorecard_xp = '//span/a[contains(@href, "/series/")]/@href'
    url_scorecard = []
    url_summary = []
    for year_url in url_year:
        # print(year_url)
        score_card_url = get_var_xpath(year_url, scorecard_xp)

        for item in score_card_url:
            url_score = url_espn + item
            url_scorecard.append(url_score)
            print(url_score)

    ########### 2.1 scorecard info
    for url in url_scorecard:
        result_sum = get_var_xpath_from_list(url, var_path.scorecard_xp)
        print(result_sum)

        test_no = result_sum[3][2]
        scorecard_no = test_no.replace('no.', '#')
        # print(scorecard_no)
        day_result = result_sum[0]

            # innings play on what day. like 1st inning play day 1, day 2, day 3
        day_ea = []
        day_country = []
        for day in day_result:
            day1 = re.split('-|innings', day)
            day2 = day1[0:2]
            if 'rest' in day2[0]:
                pass
            else:
                day_ea.append(day2[0])
                day_country.append(day2[1].strip())

            # print(innings)
        innings = []
        for item in day_country:
            # print(item)
            split = item.split()
            # print(split)
            if split[-1] == '1st' or split[-1] == '2nd':
                inns =  split[-1] + ' INNINGS'
            else:
                inns = 'no play'
            innings.append(inns)

        country_inn_testno = [item+' '+test_no for item in day_country]

        total_day = len(day_result)
        rest = [] # 1 = the day that they rest before thier inning
        for i in range(len(day_result)):
            if 'rest' in day_result[i]:
                rest_day = i+1
                rest.append(1)
            else:
                rest.append(0)
        if 1 in rest:
            rest.pop()

        toss_result = result_sum[2]
        country_bat_first = 0
        for toss in toss_result:
            if 'bat' in toss:
                bat_first = re.split(',', toss)
                country_bat_first = bat_first[0]

    scorecard_list = [scorecard_no, day_result, day_ea, day_country, innings,]




    # for item in score_card_url:
    #         new_item = item.replace('full-scorecard', 'live-cricket-score')
    #         url_sum = url_espn + new_item
    #         url_summary.append(url_sum)
    
    # print(url_scorecard)
    # print(url_summary)

    


https://www.espncricinfo.com//series/england-tour-of-australia-1876-77-61716/australia-vs-england-1st-test-62396/full-scorecard
https://www.espncricinfo.com//series/england-tour-of-australia-1876-77-61716/australia-vs-england-2nd-test-62397/full-scorecard
https://www.espncricinfo.com//series/england-tour-of-australia-1876-77-61716/australia-vs-england-1st-test-62396/full-scorecard
[['day 1 - Australia 1st innings 166/6 (Charles Bannerman 126*, Jack Blackham 3*)', 'day 2 - England 1st innings 109/4 (Harry Jupp 54*)', 'day 3 - Australia 2nd innings 83/9 (Tom Kendall 5*, John Hodges 3*)', 'rest day', 'day 4 - England 2nd innings 108 (66.1 ov) - end of match'], ['Alfred Shaw bowled the first Test delivery to Charles Bannerman, who went on to become the first Test centurion. Hill had both the first Test wicket and the first catch. Midwinter picked up the first 5 wicket haul, and Blackham had the first stumping.', 'W.Newing fielded for Bannerman.', 'Southerton is still the oldest Test debuta

### Get info from year url

In [5]:
url_year = ['https://www.espncricinfo.com/records/year/team-match-results/1877-1877/test-matches-1']

def get_sum_from_year_series(url_list):
    df_result = pd.DataFrame()

    # get infor from year
    for url in url_list:
        var = get_var_xpath_from_list(url, var_path.team1_2_xp)
        # print(var)
        info_list_ea_year = []
        for i, item in enumerate(var[1]):
            if item.startswith('Test #'):
                sublist = var[0][i * 5:(i + 1) * 5]
                sublist.append(item)
                info_list_ea_year.append(sublist)

        # print(info_list_ea_year)
        for item in info_list_ea_year:
            # print(item)
            countries = [item[0], item[1]]
            country_won = item[2]
            scorecard_no = item[-1]
            won = []    # won=1, lose=0, drawn=2
            if country_won == countries[0]:
                won = [1,0]
            elif country_won == 'drawn':
                won = [2,2]
            else:
                won = [0,1]

            dict_match_detail = {'countries': countries,
                                'team_no': [1,2],
                                'scorecard_no': scorecard_no,
                                'country_won': country_won,
                                'won': won
                                }

            df_dict = pd.DataFrame(dict_match_detail)

            df_result = pd.concat([df_result, df_dict], ignore_index=True)

    return df_result

df_from_year = get_sum_from_year_series(url_year)
df_from_year

,countries,team_no,scorecard_no,country_won,won
0,Australia,1,Test # 1,Australia,1
1,England,2,Test # 1,Australia,0
2,Australia,1,Test # 2,England,0
3,England,2,Test # 2,England,1
